In [1]:
import numpy as np
np.random.seed(0)
import robosuite as suite
import xml.etree.ElementTree as ET
import cv2
import robosuite.utils.transform_utils as T
from robosuite.utils.camera_utils import CameraMover, get_real_depth_map
from robosuite import load_controller_config

import robosuite.macros as macros

from PyLQR.sim import KDLRobot
from PyLQR.system import PosOrnPlannerSys, PosOrnKeypoint
from PyLQR.solver import BatchILQRCP, BatchILQR, ILQRRecursive
from PyLQR.utils import primitives, PythonCallbackMessage

macros.IMAGE_CONVENTION = "opencv"

render = True
controller = 'JOINT_VELOCITY' #TO DO: JOINT_VELOCITY
controller_configs = load_controller_config(default_controller=controller)

# Choose camera
camera = "frontview"
# Choose segmentation type
segmentation_level = "class" # "instance, class, or element"

# create environment instance
env = suite.make(
    env_name="Lift",
    robots="Panda",
    controller_configs=controller_configs,
    has_renderer=render,
    has_offscreen_renderer=True,
    use_camera_obs=True,
    horizon=10000000,
    render_gpu_device_id = 0,
    initialization_noise=None,
    camera_names = camera,
    control_freq =10,
    camera_depths = True,
    camera_segmentations = segmentation_level,
    camera_heights=512,
    camera_widths=512,
)


# reset the environment
env.reset()

[robosuite WARNING] No private macro file found! (__init__.py:7)
[robosuite WARNING] It is recommended to use a private macro file (__init__.py:8)
[robosuite WARNING] To setup, run: python /home/vdrame/mujoco/robosuite/robosuite/scripts/setup_macros.py (__init__.py:9)


OrderedDict([('robot0_joint_pos_cos',
              array([ 1.        ,  0.98078528,  1.        , -0.8660254 ,  1.        ,
                     -0.98006658,  0.70710678])),
             ('robot0_joint_pos_sin',
              array([ 0.        ,  0.19509032,  0.        , -0.5       ,  0.        ,
                      0.19866933,  0.70710678])),
             ('robot0_joint_vel', array([0., 0., 0., 0., 0., 0., 0.])),
             ('robot0_eef_pos',
              array([-0.10276829,  0.        ,  1.01104387])),
             ('robot0_eef_quat',
              array([9.97976606e-01, 2.45813807e-04, 6.35816977e-02, 1.56609474e-05])),
             ('robot0_gripper_qpos', array([ 0.020833, -0.020833])),
             ('robot0_gripper_qvel', array([0., 0.])),
             ('frontview_image',
              array([[[147, 144, 140],
                      [147, 144, 140],
                      [145, 142, 138],
                      ...,
                      [146, 142, 138],
                      [1

In [2]:
import colorsys
import matplotlib.cm as cm

def randomize_colors(N, bright=True):
    """
    Modified from https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/visualize.py#L59
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.5
    hsv = [(1.0 * i / N, 1, brightness) for i in range(N)]
    colors = np.array(list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv)))
    rstate = np.random.RandomState(seed=20)
    np.random.shuffle(colors)
    return colors

def segmentation_to_rgb(seg_im, random_colors=False):
    """
    Helper function to visualize segmentations as RGB frames.
    NOTE: assumes that geom IDs go up to 255 at most - if not,
    multiple geoms might be assigned to the same color.
    """
    # ensure all values lie within [0, 255]
    seg_im = np.mod(seg_im, 256)

    if random_colors:
        colors = randomize_colors(N=256, bright=True)
        return (255.0 * colors[seg_im]).astype(np.uint8)
    else:
        # deterministic shuffling of values to map each geom ID to a random int in [0, 255]
        rstate = np.random.RandomState(seed=8)
        inds = np.arange(256)
        rstate.shuffle(inds)

        # use @inds to map each geom ID to a color
        return (255.0 * cm.rainbow(inds[seg_im], 3)).astype(np.uint8)[..., :3]


In [3]:
inp = '<camera mode="fixed" name="frontview" pos="0.9980515779495429 -2.1987850487522782e-07 1.5423019978972268" quat="0.609143853187561 0.359087198972702 0.35908734798431396 0.6091439723968506" />'
from_tag = "<" in inp
cam_tree = ET.fromstring(inp) if from_tag else ET.Element("camera", attrib={"name": inp})
CAMERA_NAME = cam_tree.get("name")


# Create the camera mover
camera_mover = CameraMover(
    env=env,
    camera=CAMERA_NAME,
)

# Make sure we're using the camera that we're modifying
camera_id = env.sim.model.camera_name2id(CAMERA_NAME)
env.viewer.set_camera(camera_id=camera_id)

initial_file_camera_pos = np.array(cam_tree.get("pos").split(" ")).astype(float)
initial_file_camera_quat = T.convert_quat(np.array(cam_tree.get("quat").split(" ")).astype(float), to="xyzw")
# Set these values as well
camera_mover.set_camera_pose(pos=initial_file_camera_pos, quat=initial_file_camera_quat)


In [4]:
for i in range(1000):
    action = np.random.randn(env.robots[0].dof) # sample random action
    obs, reward, done, info = env.step(action)  # take action in the environment
    segmap = obs[f"{camera}_segmentation_{segmentation_level}"]
    print("segmap shape,", segmap.shape)
    segmap = segmap.squeeze(-1)
    print("segmap reshape,", segmap.shape)
    segmap = segmentation_to_rgb(segmap)

    depth = obs["frontview_depth"]
    depth = get_real_depth_map(env.sim, depth).squeeze(-1)
    print("depth shape", depth.shape, "\n range[{},{}]".format(depth.min(), depth.max()))
    im = obs["frontview_image"]
    cv2.imshow("segmap", segmap)
    print(depth)
    print("-------------")
    print(255*(depth-depth.min())/depth.max().astype(np.uint8))
    cv2.imshow("depth", (255*(depth-depth.min())/depth.max()).astype(np.uint8))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    cv2.imshow("im", im)
    key = cv2.waitKey(1)
    if key == ord('q'):
        env.close_renderer()
        cv2.destroyAllWindows()
        break
    #env.render()  # render on display

segmap shape, (512, 512, 1)
segmap reshape, (512, 512)
depth shape (512, 512) 
 range[0.8828471302986145,3.360642910003662]
[[2.7799964  2.7799964  2.7799964  ... 2.7799964  2.7799964  2.7799964 ]
 [2.7820385  2.7820385  2.7820385  ... 2.7820385  2.7820385  2.7820385 ]
 [2.7840402  2.7840402  2.7840402  ... 2.7840402  2.7840402  2.7840402 ]
 ...
 [0.88463724 0.88463724 0.88463724 ... 0.88463724 0.88463724 0.88463724]
 [0.8856672  0.8856672  0.8856672  ... 0.88566285 0.88566285 0.88566285]
 [0.8866952  0.8866952  0.8866952  ... 0.8866952  0.8866952  0.8866952 ]]
-------------
[[1.6125769e+02 1.6125769e+02 1.6125769e+02 ... 1.6125769e+02
  1.6125769e+02 1.6125769e+02]
 [1.6143126e+02 1.6143126e+02 1.6143126e+02 ... 1.6143126e+02
  1.6143126e+02 1.6143126e+02]
 [1.6160143e+02 1.6160143e+02 1.6160143e+02 ... 1.6160143e+02
  1.6160143e+02 1.6160143e+02]
 ...
 [1.5215904e-01 1.5215904e-01 1.5215904e-01 ... 1.5215904e-01
  1.5215904e-01 1.5215904e-01]
 [2.3970634e-01 2.3970634e-01 2.3970634e-

In [ ]:
env.close_renderer()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
from robosuite.models.objects import MujocoXMLObject

# Load the apple URDF file
apple = MujocoXMLObject("ycb/apple/model.urdf", name="apple")

In [20]:
from robosuite.models import MujocoWorldBase
from robosuite.models.robots import Panda
from robosuite.models.grippers import gripper_factory
from robosuite.models.arenas import TableArena
from robosuite.models.objects import BallObject
from robosuite.utils.mjcf_utils import new_joint
import mujoco
world = MujocoWorldBase()

mujoco_robot = Panda()
gripper = gripper_factory('PandaGripper')
mujoco_robot.add_gripper(gripper)

mujoco_robot.set_base_xpos([0, 0, 0])
world.merge(mujoco_robot)

mujoco_arena = TableArena()
mujoco_arena.set_origin([0.8, 0, 0])
world.merge(mujoco_arena)

sphere = BallObject(
    name="sphere",
    size=[0.04],
    rgba=[0, 0.5, 0.5, 1]).get_obj()
sphere.set('pos', '1.0 0 1.0')
world.worldbody.append(sphere)

model = world.get_model(mode="mujoco")


data = mujoco.MjData(model)
while data.time < 10:
    mujoco.mj_step(model, data)